In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
#Fake data
df = pd.DataFrame({})
regions = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "L"]

for i in range(len(regions)):
  df.insert(i, column=regions[i], value=np.random.laplace(size=int(1e6)))

df

,A,B,C,D,E,F,G,H,I,L
0,-1.583431,-5.690821,1.393852,-0.632255,-2.630407,-0.801062,-0.603722,4.247088,-2.706628,-0.871662
1,2.440606,1.564009,-0.306757,0.170099,0.464557,-0.527898,0.079379,-0.378123,-1.996626,0.223857
2,-0.323044,-3.833118,-0.457599,0.974531,-0.688972,-2.723195,-0.638428,0.420517,-0.522992,2.019345
3,0.073895,0.476307,-1.172540,-2.569375,0.877282,-0.273370,1.239567,0.474450,0.528610,-1.087441
4,-1.521224,0.017516,-0.060925,3.405464,0.402572,-0.167009,-1.430690,0.635711,1.167589,-0.762382
...,...,...,...,...,...,...,...,...,...,...
999995,-0.581672,-0.386457,-0.210547,0.062117,1.552010,-0.347576,-0.465095,1.268499,-0.205128,0.980443
999996,0.691099,1.152085,-0.619477,0.401816,0.426875,-2.261062,-0.232320,-0.276161,-1.918823,0.131867
999997,1.009230,-1.142433,1.508793,-0.727750,0.285071,0.007690,-0.301045,-3.682997,-1.212841,2.577396
999998,0.060582,1.020180,-0.504067,-0.437576,1.023171,-2.896188,-2.502269,1.321322,2.289088,0.147100


In [3]:
df.shape

(1000000, 10)

In [4]:
#FC
#t e tau sono indicati come indici, potrebbe servire fattore di conversione
def FC(t, tau, data):
  data_window = data[int((t-tau/2)):int((t+tau/2))]

   #Il metodo .corr() restituisce un dataframe di Pandas, .values passa ad array Numpy, lo faccio per coerenza con lo stream
   #In questo momento si sta perdendo l'info sul nome delle colonne

  FC_dataframe = data_window.corr(method="pearson")
  FC_array = FC_dataframe.values

  return FC_array

In [5]:
#dFC
def dFC(FC_1, FC_2):
  UpperTri_1 = FC_1[np.triu_indices(FC_1.shape[1], k=1)]
  UpperTri_2 = FC_2[np.triu_indices(FC_2.shape[1], k=1)]

  rho = np.corrcoef(UpperTri_1, UpperTri_2)[0, 1]
  return rho

In [6]:
#v_dFC: per definizione 1-dFC(t, t+tau)
#Si noti che si sta usando lo stesso tau per FC e per la differenza (legato a 0 overlap per dFC stream)
def v_dFC(t, tau, data):
  return  1 - dFC(FC(t, tau, data), FC(t+tau, tau, data))

In [7]:
#dFC stream 
#Si sta considerando 0 overlap tra un FC e il successivo!
def dFC_stream(data, tau):
  stream = np.zeros(shape=(1, data.shape[1], data.shape[1]))
  
  #range(10) è provvisorio, va fatto su tutto il dataset
  for i in range(10):
    stream = np.append(stream, [FC(i+tau/2, tau, data)], axis=0)
  
  return stream[1:]

In [8]:
#Dato un dFC stream calcolo tutte le v
#Nota: non viene usata v_DFC, in dFC_stream è stato imposto tau e 0 overlap
def v_stream(dFC_stream):
  v_array = np.array([])

  for i in range(dFC_stream.shape[0]-1):
    v_array = np.append(v_array, 1 - dFC(dFC_stream[i], dFC_stream[i+1]))
  
  return v_array